#Fine tuning a Pretrained model

##Dependencies

In [ ]:
!pip install huggingface-hub

In [ ]:
!huggingface-cli login

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import os
import random as rnd
!pip install evaluate
import evaluate
!pip install rouge
from rouge import Rouge
import pandas as pd

device = torch.device("cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")

device

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 42.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


device(type='cuda')

In [ ]:
!pip install -Uqq datasets transformers[torch] sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 11.0 MB/s eta 0:00:00


##Load Data

###Spanish model and Tokenizer

In [ ]:
from transformers import MarianMTModel,MarianTokenizer

model_es_zh_name = "Helsinki-NLP/opus-tatoeba-es-zh"
model_es_zh = MarianMTModel.from_pretrained(model_es_zh_name)
es_tokenizer = MarianTokenizer.from_pretrained(model_es_zh_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/155M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/835k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


###Spanish-English model and Tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_es_en_name = "Helsinki-NLP/opus-mt-es-en"
tokenizer_es_en = AutoTokenizer.from_pretrained(model_es_en_name)
model_es_en = AutoModelForSeq2SeqLM.from_pretrained(model_es_en_name)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

###English model and Tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
en_model_name = "Helsinki-NLP/opus-mt-en-zh"
en_tokenizer = AutoTokenizer.from_pretrained(en_model_name)
en_model = AutoModelForSeq2SeqLM.from_pretrained(en_model_name)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/806k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/805k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.62M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

###Spanish-Chinese

In [ ]:
data_es_zh = pd.read_csv('/content/drive/MyDrive/MTProject/Tatoeba Training/tatoeba_spa_cmn.csv')

testing_data = data_es_zh.iloc[8000:10000]

data_es_zh = data_es_zh.iloc[:7000]
data_es_zh.drop(columns=['Unnamed: 0'], inplace=True)


data_es_zh

,Spanish,Chinese
0,A Tom se le olvidó cómo hacerlo.,湯姆忘了要怎麼做。
1,A Tom se le ha olvidado cómo hacerlo.,湯姆忘了要怎麼做。
2,El invierno pasado nevó mucho.,去年冬天下了好多雪。
3,Tom le conoce.,湯姆認識他。
4,Ahora es la temporada alta de turismo.,現在是旅遊旺季。
...,...,...
6995,No trabajo sábados ni domingos.,我周六不工作，周日也不工作。
6996,Su padre no viene porque está demasiado ocupado.,她的爸爸不來因爲太忙了。
6997,El idioma nativo de Julia es el italiano.,Julia的母語是義大利語。
6998,Había una vez un enano que vivía en el bosque.,從前有個侏儒住在森林裡。


###Spanish-English

In [ ]:
import pandas as pd

data_es_en = pd.read_csv('/content/drive/MyDrive/CSV/Tatoeba Training/tatoeba_spa_eng.csv')
data_es_en = data_es_en.iloc[:7000]
data_es_en.drop(columns=['Unnamed: 0'], inplace=True)

data_es_en

,Spanish,English
0,Él le arrojó un pedazo de carne a un perro.,He threw a piece of meat to a dog.
1,No me he estado sintiendo muy bien últimamente...,"I haven't been feeling too good lately. ""You h..."
2,Él no es estúpido.,He is not stupid.
3,- ¿Sabe cuál es el prefijo del Líbano? -pregun...,Do you know the country code for Lebanon? Dima...
4,—Me parece que el prefijo del Líbano es el 961...,"I believe that the code for Lebanon is 961, th..."
...,...,...
6995,¿A qué hora te sueles acostar?,What time do you usually go to bed?
6996,"Cuanto más alto suban, más dura será la caída.","The bigger they come, the harder they fall."
6997,Hoy voy a probar esta máquina.,I am going to put this machine to the test today.
6998,La casa se colapsó en un terremoto.,The house collapsed in an earthquake.


###English-Chinese

In [ ]:
import pandas as pd

data_en_zh = pd.read_csv('/content/drive/MyDrive/CSV/Tatoeba Training/tatoeba_eng_cmn.csv')
data_en_zh = data_en_zh.iloc[:7000]
data_en_zh.drop(columns=['Unnamed: 0'], inplace=True)

data_en_zh

,English,Chinese
0,"For all the money he has, he is not happy.",儘管他有那麼多錢，他並不快樂。
1,That's the man who lives next door.,那是住在隔壁的男人。
2,Is that a picture of me?,那是我的照片嗎？
3,That is his house.,那是他家。
4,That is his car.,那輛是他的車子。
...,...,...
6995,Write your name in capital letters.,写下你名字的大写字母。
6996,Thank you ever so much.,非常感谢！
6997,I am very sorry that I have to ask you to chan...,真的很對不起，因為一些私人的理由，我想請你把見面日期改為三月六日。
6998,A big tree has fallen in the storm.,一棵大树在风暴中倒下了。


##Prepare Data

In [ ]:
# Adapted from:
# https://github.com/huggingface/notebooks/blob/main/examples/translation.ipynb
# https://github.com/guocheng98/MUTTT2020_TFM_ZGC/blob/main/HuggingFace_Fine-tune.ipynb

from datasets import Dataset
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

def prepare_data(source_language, target_language, source_data, target_data, source_tokenizer, target_tokenizer, model):
    dataset_dir = f'/content/drive/MyDrive/CSV/Tatoeba Training/{source_language}_{target_language}_dataset'

    rnd.seed(42)

    # Prepare data
    raw_data = Dataset.from_dict({source_language: source_data, target_language: target_data})
    train_test = raw_data.train_test_split(test_size=0.2, train_size=0.8)
    train_test.save_to_disk(dataset_dir)

    def preprocess_data(dataset):
        inputs = [row for row in dataset[source_language]]
        targets = [row for row in dataset[target_language]]
        model_inputs = source_tokenizer(inputs, truncation=True, max_length=512, padding="max_length")
        labels = target_tokenizer(text_target=targets, truncation=True, max_length=512, padding="max_length")
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_data = train_test.map(preprocess_data, batched=True)
    train_set = tokenized_data["train"]
    test_set = tokenized_data["test"]
    train_set.save_to_disk(f'{dataset_dir}/train')
    test_set.save_to_disk(f'{dataset_dir}/test')
    return train_set, test_set, dataset_dir


In [ ]:
def train_model(model, train_set, test_set, source_tokenizer, dataset_dir):
    checkpoint_path = f'{dataset_dir}/checkpoints/'
    os.makedirs(checkpoint_path, exist_ok=True)

    resume_from_checkpoint = None

    if len(os.listdir(checkpoint_path)) > 0:
        dirlist = os.listdir(checkpoint_path)[-1]
        finaldir = os.path.join(checkpoint_path, dirlist)
        print(finaldir)
        resume_from_checkpoint = finaldir

    training_args = Seq2SeqTrainingArguments(
        per_device_train_batch_size=8,
        num_train_epochs=3,
        output_dir=checkpoint_path,
        logging_dir=f'{dataset_dir}/logging',
        overwrite_output_dir=True,
        save_strategy='epoch',
        save_total_limit=2,
        logging_steps=100,
        eval_steps=100,
        prediction_loss_only=True
    )

    # Trainer
    trainer = Seq2SeqTrainer(
        model,
        training_args,
        train_dataset=train_set,
        eval_dataset=test_set,
        tokenizer=source_tokenizer
    )

    # Train the model
    trainer.train(resume_from_checkpoint)
    trainer.save_model(output_dir=f'{dataset_dir}/ft_checkpoints')



In [ ]:
train_set, test_set, path = prepare_data("es", "zh", data_es_zh["Spanish"], data_es_zh["Chinese"], es_tokenizer, es_tokenizer, model_es_zh)

Saving the dataset (0/1 shards):   0%|          | 0/5600 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1400 [00:00<?, ? examples/s]

Map:   0%|          | 0/5600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5600 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1400 [00:00<?, ? examples/s]

In [ ]:
train_model(model_es_zh, train_set, test_set, es_tokenizer, path)

/content/drive/MyDrive/CMPU366/CSV/Tatoeba Training/es_zh_dataset/checkpoints/checkpoint-700


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


Step,Training Loss
800,0.012100
900,0.011100
1000,0.011800
1100,0.011700
1200,0.011900
1300,0.011600
1400,0.011600
1500,0.007800
1600,0.007800
1700,0.007600


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

In [ ]:
train_set, test_set, path = prepare_data('es', 'en', data_es_en['Spanish'], data_es_en['English'], tokenizer_es_en, tokenizer_es_en, model_es_en)

Saving the dataset (0/1 shards):   0%|          | 0/5600 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1400 [00:00<?, ? examples/s]

Map:   0%|          | 0/5600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5600 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1400 [00:00<?, ? examples/s]

In [ ]:
train_model(model_es_en, train_set, test_set, tokenizer_es_en, path)

/content/drive/MyDrive/CMPU366/CSV/Tatoeba Training/es_en_dataset/checkpoints/checkpoint-2100


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


Step,Training Loss


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


In [ ]:
train_set, test_set, path = prepare_data('en', 'zh', data_en_zh['English'], data_en_zh['Chinese'], en_tokenizer, en_tokenizer, en_model)

Saving the dataset (0/1 shards):   0%|          | 0/5600 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1400 [00:00<?, ? examples/s]

Map:   0%|          | 0/5600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5600 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1400 [00:00<?, ? examples/s]

In [ ]:
train_model(en_model, train_set, test_set, en_tokenizer, path)

Step,Training Loss
100,0.028800
200,0.028700
300,0.027000
400,0.027000
500,0.026800
600,0.025700
700,0.025500
800,0.016500
900,0.015600
1000,0.015000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

##Test Output

In [ ]:
# grabs the trained models
es_zh_trained_model_link = '/content/drive/MyDrive/MTProject/Tatoeba Training/es_zh_dataset/ft_checkpoints'
es_zh_trained_model = MarianMTModel.from_pretrained(es_zh_trained_model_link)
es_zh_tokenizer = MarianTokenizer.from_pretrained(es_zh_trained_model_link)

es_en_trained_model_link = '/content/drive/MyDrive/MTProject/Tatoeba Training/es_en_dataset/ft_checkpoints'
es_en_trained_model = MarianMTModel.from_pretrained(es_en_trained_model_link)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_trained_model_link)

en_zh_trained_model_link = '/content/drive/MyDrive/MTProject/Tatoeba Training/en_zh_dataset/ft_checkpoints'
en_zh_trained_model = MarianMTModel.from_pretrained(en_zh_trained_model_link)
en_zh_tokenizer = MarianTokenizer.from_pretrained(en_zh_trained_model_link)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
# prepping testing data
testing_data.drop(columns=['Unnamed: 0'], inplace=True)
testing_data.columns = ['Spanish', 'Chinese']
testing_data.head()

,Spanish,Chinese
8000,Este colchón es muy duro.,這個床墊很硬。
8001,El sombrero es suyo.,帽子是您的。
8002,Necesito un par de zapatos nuevos.,我需要一双新鞋。
8003,¡Qué linda mujer!,好漂亮的一個女人！
8004,¿Quién es el dueño de esta casa?,屋子的主人是谁？


In [ ]:
# direct translation function from spanish to chinese
def translate_direct(input_text):
  inputs = es_zh_tokenizer(input_text, return_tensors='pt', padding=True)
  translated = es_zh_trained_model.generate(**inputs)
  translated_text = es_zh_tokenizer.decode(translated[0], skip_special_tokens=True)
  return translated_text

In [ ]:
# indirect translation function from spanish to english to chinese
def translate_indirect(input_text):
  inputs = es_en_tokenizer(input_text, return_tensors='pt', padding=True)
  en_translated = es_en_trained_model.generate(**inputs)
  en_translated_text = es_en_tokenizer.decode(en_translated[0], skip_special_tokens=True)
  inputs = en_zh_tokenizer(en_translated_text, return_tensors='pt', padding=True)
  translated = en_zh_trained_model.generate(**inputs)
  translated_text = en_zh_tokenizer.decode(translated[0], skip_special_tokens=True)
  return en_translated_text, translated_text

In [ ]:
# prepare scoring metrics
bleu_metric = evaluate.load("bleu")

In [ ]:
# helper function that adds a white space between each chinese character
def split_chinese(chinese_text):
  formatted_text = ' '.join(ch for ch in chinese_text)
  return formatted_text

In [ ]:
# helper function that looks at the rouge score
def get_rouge_score(predictions, references):
  score = rouge.get_scores(predictions, references)
  return score

In [ ]:
# helper function that looks at the bleu score
def get_bleu_score(predictions, references):
  score = bleu_metric.compute(predictions=predictions, references=references)
  return score['bleu'], score['precisions'], score['brevity_penalty'], score['length_ratio'], score['translation_length'], score['reference_length']

In [ ]:
direct_csv = {}
direct_headers = ['Spanish Text', 'Chinese Text', 'Translated Text', ]
newIndex = 0
for index, row in testing_data.iterrows():

  # grab row data
  spanish_text = row['Spanish']
  chinese_text = row['Chinese']

  # prep chinese text
  chinese_text = split_chinese(chinese_text)

  # get translated text and prep
  translated_text = translate_direct(spanish_text)
  translated_text = split_chinese(translated_text)

  # get bleu score
  predictions = [translated_text]
  references = [chinese_text]
  bleu_score, precisions, brevity, length_ratio, trans_length, ref_length = get_bleu_score(predictions, references)

  # add to dictionary
  direct_csv[newIndex] = [spanish_text, chinese_text, translated_text, bleu_score, precisions, brevity, length_ratio, trans_length, ref_length]
  newIndex += 1

  print(f'{newIndex}/2000 done!')

# convert dictionary to dataframe
direct_df = pd.DataFrame.from_dict(direct_csv, orient='index', columns=direct_headers)
direct_df.to_csv('/content/drive/MyDrive/MTProject/Tatoeba Testing/direct_translation.csv')

In [ ]:
indirect_csv = {}
indirect_headers = ['Spanish Text', 'Chinese Text', 'English Text', 'Translated Text', 'Bleu Score', 'Precisions', 'Brevity Penalty', 'Length Ratio', 'Translation Length', 'Reference Length']
newIndex = 0
for index, row in testing_data.iterrows():

  # grab row data
  spanish_text = row['Spanish']
  chinese_text = row['Chinese']

  # prep chinese text
  chinese_text = split_chinese(chinese_text)

  # get translated text
  trans_english_text, trans_chinese_text = translate_indirect(spanish_text)
  trans_chinese_text = split_chinese(trans_chinese_text)

  # get bleu score
  predictions = [trans_chinese_text]
  references = [chinese_text]
  bleu_score, precisions, brevity, length_ratio, trans_length, ref_length = get_bleu_score(predictions, references)

  # add to dictionary
  indirect_csv[newIndex] = [spanish_text, chinese_text, trans_english_text, trans_chinese_text, bleu_score, precisions, brevity, length_ratio, trans_length, ref_length]
  newIndex += 1

  print(f'{newIndex}/2000 done!')

# convert dictionary to dataframe
indirect_df = pd.DataFrame.from_dict(indirect_csv, orient='index', columns=indirect_headers)
indirect_df.to_csv('/content/drive/MyDrive/MTProject/Tatoeba Testing/indirect_translation.csv')

In [ ]:
# grab csv files of testing results
indirect_test = pd.read_csv('/content/drive/MyDrive/MTProject/Tatoeba Testing/indirect_translation.csv')
direct_test = pd.read_csv('/content/drive/MyDrive/MTProject/Tatoeba Testing/direct_translation.csv')
print(indirect_test.columns.tolist())
print(direct_test.columns.tolist())

['Unnamed: 0', 'Spanish Text', 'Chinese Text', 'English Text', 'Translated Text', 'Bleu Score', 'Precisions', 'Brevity Penalty', 'Length Ratio', 'Translation Length', 'Reference Length']
['Unnamed: 0', 'Spanish Text', 'Chinese Text', 'Translated Text', 'Bleu Score', 'Precisions', 'Brevity Penalty', 'Length Ratio', 'Translation Length', 'Reference Length']


In [ ]:
# direct bleu results
direct_avg_bleu = round(direct_test['Bleu Score'].mean(), 3)
direct_avg_length = round(direct_test['Length Ratio'].mean(), 3)

# indirect bleu results
indirect_avg_bleu = round(indirect_test['Bleu Score'].mean(), 3)
indirect_avg_length = round(indirect_test['Length Ratio'].mean(), 3)

# print out results
total_width = 92
print("Comparison of BLEU Scores for Direct and Indirect Translations")
print("--------------------------------------------------------------------------------------------")
print(f"{'Average BLEU Score For Direct Translations'.ljust(total_width//2)}{(str(direct_avg_bleu)).rjust(total_width//2)}")
print(f"{'Average Length Ratio For Direct Translations'.ljust(total_width//2)}{(str(direct_avg_length)).rjust(total_width//2)}")
print(f"{'Average BLEU Score For Indirect Translations'.ljust(total_width//2)}{(str(indirect_avg_bleu)).rjust(total_width//2)}")
print(f"{'Average Length Ratio For Indirect Translations'.ljust(total_width//2)}{(str(indirect_avg_length)).rjust(total_width//2)}")
print("--------------------------------------------------------------------------------------------")

Comparison of BLEU Scores for Direct and Indirect Translations
--------------------------------------------------------------------------------------------
Average BLEU Score For Direct Translations                                             0.255
Average Length Ratio For Direct Translations                                           0.996
Average BLEU Score For Indirect Translations                                           0.244
Average Length Ratio For Indirect Translations                                          1.08
--------------------------------------------------------------------------------------------


In [ ]:
direct_csv_rouge = {}
direct_headers_rouge = ['Spanish Text', 'Chinese Text', 'Translated Text', 'Rouge-1 F1', 'Rouge-1 Precision', 'Rouge-1 Recall', 'Rouge-2 F1', 'Rouge-2 Precision', 'Rouge-2 Recall', 'Rouge-l F1', 'Rouge-l Precision', 'Rouge-l Recall']
newIndex = 0
for index, row in testing_data.iterrows():

  # grab row data
  spanish_text = row['Spanish']
  chinese_text = row['Chinese']

  # prep chinese text
  chinese_text = split_chinese(chinese_text)

  # get translated text and prep
  translated_text = translate_direct(spanish_text)
  translated_text = split_chinese(translated_text)

  # getrouge score
  predictions = [translated_text]
  references = [chinese_text]
  rouge = Rouge()
  rouge_score =  get_rouge_score(predictions, references)[0]
  rouge_1_f1 = rouge_score['rouge-1']['f']
  rouge_1_precision = rouge_score['rouge-1']['p']
  rouge_1_recall = rouge_score['rouge-1']['r']
  rouge_2_f1 = rouge_score['rouge-2']['f']
  rouge_2_precision = rouge_score['rouge-2']['p']
  rouge_2_recall = rouge_score['rouge-2']['r']
  rouge_l_f1 = rouge_score['rouge-l']['f']
  rouge_l_precision = rouge_score['rouge-l']['p']
  rouge_l_recall = rouge_score['rouge-l']['r']

  # add to dictionary
  direct_csv_rouge[newIndex] = [spanish_text, chinese_text, translated_text, rouge_1_f1, rouge_1_precision, rouge_1_recall, rouge_2_f1, rouge_2_precision, rouge_2_recall, rouge_l_f1, rouge_l_precision, rouge_l_recall]
  newIndex += 1

  print(f'{newIndex}/2000 done!')

# convert dictionary to dataframe
direct_df_rouge = pd.DataFrame.from_dict(direct_csv_rouge, orient='index', columns=direct_headers_rouge)
direct_df_rouge.to_csv('/content/drive/MyDrive/MTProject/Tatoeba Testing/direct_translation_rouge.csv')

In [ ]:
indirect_csv_rouge = {}
indirect_headers_rouge = ['Spanish Text', 'Chinese Text', 'English Text', 'Translated Text', 'Rouge-1 F1', 'Rouge-1 Precision', 'Rouge-1 Recall', 'Rouge-2 F1', 'Rouge-2 Precision', 'Rouge-2 Recall', 'Rouge-l F1', 'Rouge-l Precision', 'Rouge-l Recall']
newIndex = 0
for index, row in testing_data.iterrows():

  # grab row data
  spanish_text = row['Spanish']
  chinese_text = row['Chinese']

  # prep chinese text
  chinese_text = split_chinese(chinese_text)

  # get translated text
  trans_english_text, trans_chinese_text = translate_indirect(spanish_text)
  trans_chinese_text = split_chinese(trans_chinese_text)

  # get rouge score
  predictions = [trans_chinese_text]
  references = [chinese_text]
  rouge = Rouge()
  rouge_score =  get_rouge_score(predictions, references)[0]
  rouge_1_f1 = rouge_score['rouge-1']['f']
  rouge_1_precision = rouge_score['rouge-1']['p']
  rouge_1_recall = rouge_score['rouge-1']['r']
  rouge_2_f1 = rouge_score['rouge-2']['f']
  rouge_2_precision = rouge_score['rouge-2']['p']
  rouge_2_recall = rouge_score['rouge-2']['r']
  rouge_l_f1 = rouge_score['rouge-l']['f']
  rouge_l_precision = rouge_score['rouge-l']['p']
  rouge_l_recall = rouge_score['rouge-l']['r']

  # add to dictionary
  indirect_csv_rouge[newIndex] = [spanish_text, chinese_text, trans_english_text, trans_chinese_text, rouge_1_f1, rouge_1_precision, rouge_1_recall, rouge_2_f1, rouge_2_precision, rouge_2_recall, rouge_l_f1, rouge_l_precision, rouge_l_recall]
  newIndex += 1

  print(f'{newIndex}/2000 done!')

# convert dictionary to dataframe
indirect_df_rouge = pd.DataFrame.from_dict(indirect_csv_rouge, orient='index', columns=indirect_headers_rouge)
indirect_df_rouge.to_csv('/content/drive/MyDrive/MTProject/Tatoeba Testing/indirect_translation_rouge.csv')

In [ ]:
indirect_bleu = round(indirect_test['Bleu Score'].mean(), 3)
direct_bleu = round(direct_test['Bleu Score'].mean(), 3)
print(f'Indirect Bleu Score: {indirect_bleu}')
print(f'Direct Bleu Score: {direct_bleu}')

NameError: name 'indirect_test' is not defined

In [ ]:
# grab csv files of testing results
indirect_test_rouge = pd.read_csv('/content/drive/MyDrive/MTProject/Tatoeba Testing/indirect_translation_rouge.csv')
direct_test_rouge = pd.read_csv('/content/drive/MyDrive/MTProject/Tatoeba Testing/direct_translation_rouge.csv')
print(indirect_test_rouge.columns.tolist())
print(direct_test_rouge.columns.tolist())

['Unnamed: 0', 'Spanish Text', 'Chinese Text', 'English Text', 'Translated Text', 'Rouge-1 F1', 'Rouge-1 Precision', 'Rouge-1 Recall', 'Rouge-2 F1', 'Rouge-2 Precision', 'Rouge-2 Recall', 'Rouge-l F1', 'Rouge-l Precision', 'Rouge-l Recall']
['Unnamed: 0', 'Spanish Text', 'Chinese Text', 'Translated Text', 'Rouge-1 F1', 'Rouge-1 Precision', 'Rouge-1 Recall', 'Rouge-2 F1', 'Rouge-2 Precision', 'Rouge-2 Recall', 'Rouge-l F1', 'Rouge-l Precision', 'Rouge-l Recall']


In [ ]:
# direct rouge results
direct_avg_1_f1 = round(direct_test_rouge['Rouge-1 F1'].mean(), 3)
direct_avg_1_pre = round(direct_test_rouge['Rouge-1 Precision'].mean(), 3)
direct_avg_1_re = round(direct_test_rouge['Rouge-1 Recall'].mean(), 3)
direct_avg_2_f1 = round(direct_test_rouge['Rouge-2 F1'].mean(), 3)
direct_avg_2_pre = round(direct_test_rouge['Rouge-2 Precision'].mean(), 3)
direct_avg_2_re = round(direct_test_rouge['Rouge-2 Recall'].mean(), 3)
direct_avg_l_f1 = round(direct_test_rouge['Rouge-l F1'].mean(), 3)
direct_avg_l_pre = round(direct_test_rouge['Rouge-l Precision'].mean(), 3)
direct_avg_l_re = round(direct_test_rouge['Rouge-l Recall'].mean(), 3)

# indirect rouge results
indirect_avg_1_f1 = round(indirect_test_rouge['Rouge-1 F1'].mean(), 3)
indirect_avg_1_pre = round(indirect_test_rouge['Rouge-1 Precision'].mean(), 3)
indirect_avg_1_re = round(indirect_test_rouge['Rouge-1 Recall'].mean(), 3)
indirect_avg_2_f1 = round(indirect_test_rouge['Rouge-2 F1'].mean(), 3)
indirect_avg_2_pre = round(indirect_test_rouge['Rouge-2 Precision'].mean(), 3)
indirect_avg_2_re = round(indirect_test_rouge['Rouge-2 Recall'].mean(), 3)
indirect_avg_l_f1 = round(indirect_test_rouge['Rouge-l F1'].mean(), 3)
indirect_avg_l_pre = round(indirect_test_rouge['Rouge-l Precision'].mean(), 3)
indirect_avg_l_re = round(indirect_test_rouge['Rouge-l Recall'].mean(), 3)

# print out results
total_width = 120
print("Comparison of ROUGE Scores for Direct and Indirect Translations")
print("------------------------------------------------------------------------------------------------------------------------")
print(f"{'Average Rouge-1 F1 Score For Direct Translations'.ljust(total_width//2)}{(str(direct_avg_1_f1)).rjust(total_width//2)}")
print(f"{'Average Rouge-1 Precision Score For Direct Translations'.ljust(total_width//2)}{(str(direct_avg_1_pre)).rjust(total_width//2)}")
print(f"{'Average Rouge-1 Recall Score For Direct Translations'.ljust(total_width//2)}{(str(direct_avg_1_re)).rjust(total_width//2)}")
print(f"{'Average Rouge-2 F1 Score For Direct Translations'.ljust(total_width//2)}{(str(direct_avg_2_f1)).rjust(total_width//2)}")
print(f"{'Average Rouge-2 Precision Score For Direct Translations'.ljust(total_width//2)}{(str(direct_avg_2_pre)).rjust(total_width//2)}")
print(f"{'Average Rouge-2 Recall Score For Direct Translations'.ljust(total_width//2)}{(str(direct_avg_2_re)).rjust(total_width//2)}")
print(f"{'Average Rouge-l F1 Score For Direct Translations'.ljust(total_width//2)}{(str(direct_avg_l_f1)).rjust(total_width//2)}")
print(f"{'Average Rouge-l Precision Score For Direct Translations'.ljust(total_width//2)}{(str(direct_avg_l_pre)).rjust(total_width//2)}")
print(f"{'Average Rouge-l Recall Score For Direct Translations'.ljust(total_width//2)}{(str(direct_avg_l_re)).rjust(total_width//2)}")
print()
print(f"{'Average Rouge-1 F1 Score For Indirect Translations'.ljust(total_width//2)}{(str(indirect_avg_1_f1)).rjust(total_width//2)}")
print(f"{'Average Rouge-1 Precision Score For Indirect Translations'.ljust(total_width//2)}{(str(indirect_avg_1_pre)).rjust(total_width//2)}")
print(f"{'Average Rouge-1 Recall Score For Indirect Translations'.ljust(total_width//2)}{(str(indirect_avg_1_re)).rjust(total_width//2)}")
print(f"{'Average Rouge-2 F1 Score For Indirect Translations'.ljust(total_width//2)}{(str(indirect_avg_2_f1)).rjust(total_width//2)}")
print(f"{'Average Rouge-2 Precision Score For Indirect Translations'.ljust(total_width//2)}{(str(indirect_avg_2_pre)).rjust(total_width//2)}")
print(f"{'Average Rouge-2 Recall Score For Indirect Translations'.ljust(total_width//2)}{(str(indirect_avg_2_re)).rjust(total_width//2)}")
print(f"{'Average Rouge-l F1 Score For Indirect Translations'.ljust(total_width//2)}{(str(indirect_avg_l_f1)).rjust(total_width//2)}")
print(f"{'Average Rouge-l Precision Score For Indirect Translations'.ljust(total_width//2)}{(str(indirect_avg_l_pre)).rjust(total_width//2)}")
print(f"{'Average Rouge-l Recall Score For Indirect Translations'.ljust(total_width//2)}{(str(indirect_avg_l_re)).rjust(total_width//2)}")
print("------------------------------------------------------------------------------------------------------------------------")

Comparison of ROUGE Scores for Direct and Indirect Translations
------------------------------------------------------------------------------------------------------------------------
Average Rouge-1 F1 Score For Direct Translations                                                                   0.658
Average Rouge-1 Precision Score For Direct Translations                                                            0.669
Average Rouge-1 Recall Score For Direct Translations                                                               0.656
Average Rouge-2 F1 Score For Direct Translations                                                                   0.413
Average Rouge-2 Precision Score For Direct Translations                                                            0.419
Average Rouge-2 Recall Score For Direct Translations                                                               0.411
Average Rouge-l F1 Score For Direct Translations                                         